In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [3]:
test_data_path = "tests/mar-14/02-critical-compliances.txt"
with open(test_data_path, 'r') as file:
    test_data = file.readlines()
test_data = [line.strip() for line in test_data]
test_data = [line for line in test_data if line]
# remove 'CRITICAL:' from the start of each line
test_data = [line[10:] if line.startswith("CRITICAL:") else line for line in test_data]
test_data[:5]

['The purpose of this Request for Proposal is to award an Indefinite Delivery Indefinite Quantity (IDIQ) contract for On-Demand Coaching Services for the Treasury Executive Institute. The Period of Performance is from 9/30/2023 to 9/29/2027, with one 12-month base period and three 12-month option periods.',
 'This is a combined synopsis/solicitation for commercial products or commercial services prepared in accordance with the format in subpart 12.6, as supplemented with additional information included in this notice. Solicitation 2032H323R00065 is issued as a request for proposal (RFP) for a single award Indefinite Delivery Indefinite Quantity (IDIQ) contract consistent with Federal Acquisition Regulations (FAR) part 12, and 13 as appropriate. All Task Orders issued from resultant IDIQ contracts will be awarded using the procedures stated at FAR16.505 and are subject to the terms and conditions outlined. The minimum guarantee for this acquisition is $57,000.00.',
 'Individual task ord

In [5]:
from llama_index.core import SimpleDirectoryReader
data_directory = "tests/mar-15/data/"
reader = SimpleDirectoryReader(data_directory)
documents = reader.load_data()

In [15]:
from llama_index.core.node_parser import SentenceSplitter
nodes = SentenceSplitter(chunk_size=256).get_nodes_from_documents(documents)

In [17]:
from IPython.display import display, Markdown
display(Markdown(nodes[0].get_content()))

Department of Treasury Request for Proposal Response 

Solicitation # 2032H323R00065

On-Demand Executive Coaching with Assessments &

Leadership Development Resources



Volume III: Management Approach



Submitted By:

Company Name: Sagely, LLC

Address: 320 South Main Street Providence, RI 

DUNS: 075888104



Point of Contact: Mandy Graham

Phone Number: 781.626.2797

Email: mandy@growsagely.com



SUBMITTED TO:

Mr. Aaron C. Hill & Ms. Angela Gallo

5000 ELLIN ROAD

Lanham MD 20706

In [18]:
from govrfp.rfpinsights._chromadb import return_collection, add_entries_to_collection
collection_path = "tests/mar-15/chroma/"
collection = return_collection(collection_path, "comparition_test")
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [23]:
nodes[0].__dict__

{'id_': 'da371d22-0735-4929-99b7-824c937c3534',
 'embedding': None,
 'metadata': {'file_name': 'Sagely Management Approach RFP.docx',
  'file_path': '/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp/tests/mar-15/data/Sagely Management Approach RFP.docx',
  'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'file_size': 1139130,
  'creation_date': '2024-03-15',
  'last_modified_date': '2024-03-14'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6c28c4f2-0d5f-4e68-b35b-9e1c07438a5d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'Sagely Management Approach RFP.docx', 'file_path': '/Users/chinmayshrivastava/Documents/G

In [24]:
# add all the nodes to the collection
docs = [node.get_content() for node in nodes]
metadata = [node.metadata for node in nodes]
ids = [node.id_ for node in nodes]
if collection.count() == 0:
    add_entries_to_collection(docs=docs, metadata=metadata, ids=ids, collection=collection)

Collection(name=comparition_test)

In [25]:
collection.count()

2792

In [32]:
PROMPT_TEMPLATE = (
    "For the given compliance related to a certain RFP, and excerpts from the submission to that RFP, return 1 is the compliance is met, 0 if not met, and -1 if the compliance is not applicable. \n"
    "Compliance: \n"
    "{compliance}\n"
    "Submission: \n"
    "{submission}\n"
    "Return only 1, 0, or -1 based on the submission.\n"
    "Response:"
)
print(PROMPT_TEMPLATE)

For the given compliance related to a certain RFP, and excerpts from the submission to that RFP, return 1 is the compliance is met, 0 if not met, and -1 if the compliance is not applicable. 
Compliance: 
{compliance}
Submission: 
{submission}
Return only 1, 0, or -1 based on the submission.
Response:


In [36]:
import tqdm
responses = []
for compliance in tqdm.tqdm(test_data):
    prompt = PROMPT_TEMPLATE.format(compliance=compliance, submission="\n".join([f"{document}" for i, document in enumerate(collection.query(query_texts=[compliance], n_results=3)["documents"][0])]))
    response = llm.complete(prompt).text
    responses.append(response)

100%|██████████| 115/115 [01:28<00:00,  1.31it/s]


In [35]:
responses

['-1', '0', '1', '0', '1', '1', '1']